<a href="https://colab.research.google.com/github/kmc0207/Tarot/blob/master/dataclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
!pip install pubchempy
from pubchempy import get_compounds
import pubchempy as pcp
from urllib.request import urlopen
import os
import re
import lxml
import time
import shutil
import warnings
import requests
from tqdm import tqdm
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.request import urlopen

target_page = ["https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html"]
source_dir = "FAERSsrc"
data_dir = "FAERSdata"
warnings.filterwarnings('ignore')

  Created wheel for pubchempy: filename=PubChemPy-1.0.4-cp36-none-any.whl size=13827 sha256=79aaa68e6aa33c40b0d266026ea2ad8223259b1b0a145d3c2ffcd97862253690
  Stored in directory: /root/.cache/pip/wheels/10/4d/51/6b843681a9a5aef35f0d0fbce243de46f85080036e16118752
Successfully built pubchempy


현성이에게

일단은 아래의 함수들을 한번 전부 컴파일 해줘,

1. data 받는 법 :
  DataDownMain() 함수를 작동시키면 너의 지금 위치(pwd기준)에 FAERSsrc와 FAERSdata 폴더를 만들고, 그 중에서도 FAERSsrc/ASCII 파일에 모든 데이터를 저장해줄거야. 내가 처리하는 Data는 DRUG, 네가 처리할 Data는 REAC일거야

2. Data 읽는 법 : 
  다운받아진 txt 파일을 pd.read_csv("파일위치",sep='$')으로 읽어오면 데이터프레임 형식으로 데이터를 읽을 수 있을 거야

3. 전처리 하는 법 :
  DRUG파일을 하나 읽어서(여러개 merge해도 되긴 한데 실험은 안해봤어, 시간이 엄청 오래 걸릴 거야) All_of_pre 함수에 넣고 output을 받으면 전부 전처리 되어 있을 거야. 에러가 생기면 나에게 말하거나 직접 해결해도 괜찮아


인턴 일 화이팅이야!



https://www.fda.gov/drugs/questions-and-answers-fdas-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files 
데이터 파일을 하나씩 받고 싶다면 저기서 ASCII라고 적힌 파일을 받으면 돼.

여기서부터는 읽지 않아도 괜찮은 부분인데, 전처리 과정을 간단히라도 알려줄게. 그래야 문제가 생기거나 수정할때 더 쉬울 테니까.

일단은 중복 제거나 Nan, Null 값 제거를 하고, route가 Oral이 아닌 데이터를 전부 지워.

그리고 나서 PS가 없는 caseid들을 전부 지우고, 같은 role_cod를 가지면서 prod_ai가 같은 값을 중복 제거해. 그 이후에 한 caseid당 하나밖에 컴포넌트가 없는 경우를 전부 지우고, PID와 SMiles를 찾아서 추가해줘. 이게 전부야


마지막으로, REAC 파일과 DRUG 파일은 합칠 수 없어. 왜냐면 REAC 파일은 하나의 caseid당 여러 개의 side effect를 기술해두었지만, DRUG 파일은 sied effect 갯수와는 아예 무관하게 약물을 전부 적어두었기 때문에 합치기는 쉽지 않을 것 같아.

내 생각에는 REAC 파일을 따로 처리 하고 둘을 caseid에 맞추어서 합치는게 합리적일것 같아. 혹시 하다가 이해가 안되는 부분이 있거나 에러가 있거나 나의 도움이 필요하면 언제든지 카톡해줘!

In [2]:
def getFilesUrl():
    """
    find all web urls in target page.
    :return: dict files = {"name":"url"}
    """
    print("Get web urls.\t")
    files = {}
    for page_url in target_page:
        try:
            request = urlopen(page_url)
            page_bs = BeautifulSoup(request, "lxml")
            request.close()
        except:
            request = urlopen(page_url)
            page_bs = BeautifulSoup(request)
        for url in page_bs.find_all("a"):
            a_string = str(url)
            if "ASCII" in a_string.upper():
                t_url = url.get('href')
                files[str(url.get('href'))[-16:-4]] = t_url

    # save urls to FaersFilesWebUrls.txt
    save_path = os.getcwd() + "/FaersFilesWebUrls.txt"
    if (os.path.exists(save_path)):
        os.remove(save_path)
    with open(save_path, 'a+') as f:
        for k in files.keys():
            f.write(k + ":" + files[k] + "\n")

    print("Done!")
    return files

In [3]:
def downloadFiles(faers_files, source_dir, data_dir):
    """
    download faers data files.
    :param faers_files: dict faers_files = {"name":"url"}
    :param source_dir: FAERSsrc
    :param data_dir: FAERSdata
    :return: none
    """
    for file_name in tqdm(faers_files):
        try:
            print("Download " + file_name + "\t" + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            r = requests.get(faers_files[file_name], timeout=200)
            z = ZipFile(BytesIO(r.content))
            z.extractall(source_dir)
            r.close()

            # delete and copy files to FAERSdata.
            deleteUnwantedFiles(source_dir)
            copyFiles(source_dir, data_dir)
            print("Download " + file_name + " success!\t" + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        except Exception as e:
            print("Download " + file_name + " failed! Error: " + str(e))
        print("Sleep 30 seconds before starting download a new file.\n")
        time.sleep(30)

In [4]:
def DataDownMain():
    # creating the source directory if not exits.
    if not os.path.isdir(source_dir):
        os.makedirs(source_dir)
    if not os.path.isdir(data_dir):
        os.makedirs(data_dir)

    # get faers data file's url and download them.
    faers_files = getFilesUrl()
    downloadFiles(faers_files, source_dir, data_dir)

In [5]:

def deleteUnwantedFiles(path):
    """
    delete unwanted files.
    :param path: FAERSsrc
    :return: none
    """
    print("Delete unwanted files.\t" + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    for parent, dirnames, filenames in os.walk(path):
        for fn in filenames:
            # FDA Adverse Event Reporting System (FAERS) began 2012Q4.
            # keep data from 2012Q4 and after.
            if fn[4:8] < "12Q4":
                print("Delete " + fn)
                os.remove(os.path.join(parent, fn))
            if fn.lower().endswith('.pdf') or fn.lower().endswith('.doc'):
                print("Delete " + fn)
                os.remove(os.path.join(parent, fn))
            elif fn.upper().startswith(("RPSR", "INDI", "THER", "SIZE", "STAT", "OUTC")):
                print("Delete " + fn)
                os.remove(os.path.join(parent, fn))
    print("Delete unwanted files done!\t" + datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
def read_file():
  #file을 읽어오는 함수, Colab에 올라가 있는 것 위주로 작성되었으니 한번에 다운받아 합치기는 구현해야함
  return pd.read_csv("/content/drive/My Drive/Colab Notebooks/DRUG20Q1.txt",low_memory=False,sep='$',error_bad_lines=False)

In [7]:
def preprocessing(df):
  # input : dataframe
  # output : 'caseid','role_cod','drugname','prod_ai','route' 기준으로 중복 제거
  #          null값 제거
  #          route = Oral이 아닌 것들 전부 제거  
  #          한 dataframe
  
  K = df.drop_duplicates(['caseid','role_cod','drugname','prod_ai','route'])
  print('drop duplicates in role_cod,drugname,prodai,route,caseid....')
  idx_nm = K[(K['route']!='Oral')].index
  Q = K.drop(idx_nm)
  print('drop all not-Oral component.....')
  W = Q.drop(['drug_seq','val_vbm','dose_vbm','cum_dose_chr', 'cum_dose_unit', 'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt', 'dose_unit', 'dose_form', 'dose_freq'],axis='columns')
  print('drop dosage imformation....')
  W = W.dropna(axis=0)
  print('drop all null data....')
  C = W.drop_duplicates(['caseid','prod_ai'])
  print('drop duplicates in caseid and prodai....')
  return C

In [8]:
def delete_duplicate(df):
  # input : dataframe (preprocessing 이후)
  # output : 같은 caseid, role_cod, prod_ai이면 제거
  #          role_cod는 'SS'와 'PS'를 같은 것으로 취급
  #          한 dataframe
  # 부가설명 : 같은 caseid, role_cod, proa_ai이면 비슷한 약물이나 복용량(dosage)에서 차이가 난 것이기 때문에 다 제거해서 하나로 남겨준다. PS와 SS가 같을 경우 꼭 PS만 남겨준다!
  C = df[df['role_cod']=='C']
  S = df[(df['role_cod']=='PS')|(df['role_cod']=='SS')]
  I = df[df['role_cod']=='I']
  F = pd.concat([S.drop_duplicates(subset=['caseid','prod_ai']),C.drop_duplicates(subset=['caseid','prod_ai']),I.drop_duplicates(subset=['caseid','prod_ai'])])
  print('drop duplicates in caseid, role_cod, prod_ai....')
  return F.sort_values(by=['caseid'])

In [9]:
def delete_only_C(df):
  # input : dataframe (delete_duplicat이후)
  # output: PS가 없는 caseid를 전부 제외한 datframe
  ps_id = df[df['role_cod']=='PS']['caseid'].unique()
  all_id = df['caseid'].unique()
  ps_id_set =set(ps_id)
  all_id_set = set(all_id)
  ps_id = list(ps_id_set)
  all_id = list(all_id_set)
  delete_set = all_id_set - ps_id_set
  delete_list = list(delete_set)
  print("drop all caseid which don't have PS")
  df.drop(df.loc[df['caseid'].isin(delete_list)].index,inplace=True)
  return df    

In [10]:
def find_only_one_row(df):
  # input : dataframe(delete_only_c 진행 이후)
  # output : OO(dataframe with one row), NO(dataframe with more than one row)
  #          input에서 caseid당 component가 한 줄인 것과 아닌 것을 나눠주는 함수
  #          시간이 좀 걸릴 수 있다.
  print('find more than one row....')
  grouped = df.groupby(df['caseid'])
  Only_one= []
  not_one=[]
  for caseid,group_data in grouped:
    if (group_data[:].shape[0]) == 1 :
      Only_one.append(group_data[:])
    else :
      not_one.append(group_data[:])
  OO = pd.concat(Only_one)
  NO = pd.concat(not_one)
  return OO,NO

In [11]:
def prodai_to_pupchem(df):
  # input : 앞의 함수가 모두 적용된 dataframe
  # output : coumpound의 cid와 smiles가 추가된 dataframe
  names = df['prod_ai']
  unique_name = df['prod_ai'].unique()
  dict_of_drug = {}
  CID =[]
  SMILES= []
  none= 0
  i=0
  j=0
  for n in unique_name :
    if (i/len(unique_name))*100 % 10 == 0:
      print("dict",i*100/len(unique_name))
    i = i+1
    cid = []
    smiles =[]
    cid_t = ''
    smiles_t=''
    n_name = n.split('\\')
    for name in n_name :
      print(name)
      compounds = pcp.get_compounds(name,'name')
      for compound in compounds :
          cid.append(str(compound.cid))
          smiles.append(compound.isomeric_smiles)
    k=0   
    for cids in cid:
      if k ==0 :
        cid_t = cids
      else :
        cid_t =  cid_t+', '+cids
      k=k+1
    k=0
    for smileses in smiles:
      if k == 0:
        smiles_t = smileses
      else :
        smiles_t = smiles_t+', '+smileses
      k=k+1
    print(cid)
    dict_of_drug[n] = [cid_t,smiles_t]
  for name in names :
    if (j/len(names))*100 % 10 == 0:
      print("dict",i*100/len(names))
    j=j+1
    CID.append(dict_of_drug[name][0])
    SMILES.append(dict_of_drug[name][1])
  CID_df = pd.DataFrame({"CID":CID})
  SMILES_df =pd.DataFrame({"smiles":SMILES})
  NO_index = df.reset_index()
  NO_p = pd.DataFrame({'caseid' : NO_index['caseid'],'role_cod':NO_index['role_cod'],'drugname':NO_index['drugname'],'prod_ai':NO_index['prod_ai'],'route':NO_index['route'],'CID':CID_df['CID'],'SMILES':SMILES_df['smiles']})
  NO_p = NO_p.dropna(axis=0)
  return NO_p,dict_of_drug

In [12]:
def prod_ai_to_pub_duplicate(df):
  # input : 전처리가 끝난 dataframe
  # output : 각 prod_ai의 성분들의 cid와 smiles가 한줄씩 나눠져서 들어가있는 dataframe
  # 시간이 좀 걸릴 수 있음
  # pubchem server busy가 걸릴 수 있는데 그러면 다시 해야함
  # dict 뒤에 숫자는 dictionary 만들기의 %를 뜻한다
  print('prod_ai to pid and smiles....')
  print(df)
  i=0;
  pub = []
  u_name =[]
  for name in df['prod_ai']:
    n = name.split('\\')
    for ns in n:
      u_name.append(ns)
  dict_of_drug = {}
  u_name=set(u_name)
  u_name=list(u_name)
  for n in u_name:
    if ( (i /len(u_name))*100%10==0):
      print("dict",i*100/len(u_name))
    i=i+1
    CID = []
    SMILES= []
    compounds = pcp.get_compounds(n,'name')
    for compound in compounds:
       CID.append(str(compound.cid))
       SMILES.append(compound.isomeric_smiles)
    dict_of_drug[n]=[CID,SMILES]
    time.sleep(1)
  j=0
  for index,row in df.iterrows():
    if ( j == 0):
      j=j+1
      continue
    j=j+1
    if (j/len(df)*100%10==0):
      print(j/len(df)*100)
    n_name = row['prod_ai'].split('\\')
    for name in n_name:
      cids = dict_of_drug[name][0]
      smiles = dict_of_drug[name][1]
      for k in range(0,len(cids)):
        multi=[]
        multi.append(row['caseid'])
        multi.append(row['role_cod'])
        multi.append(row['drugname'])
        multi.append(name)
        multi.append(cids[k])
        multi.append(smiles[k])
        pub.append(multi)
  return pub

In [13]:
def All_of_pre(df):
  #이거에 df를 넣고 엔터만 치면 된다!
  df1 = preprocessing(df)
  df2 = delete_duplicate(df1)
  df3 = delete_only_C(df2)
  OO,df4 = find_only_one_row(df3)
  df5 = prod_ai_to_pub_duplicate(df4)
  new_pd = pd.DataFrame(df5)
  new=pd.DataFrame({'caseid':new_pd[0],'role_cod':new_pd[1],'drugname':new_pd[2],'prod_ai':new_pd[3],'CID':new_pd[4],'SMILES':new_pd[5]})
  return new

In [14]:
df= read_file()


In [16]:
df2 = preprocessing(df)
print(df2)

drop duplicates in role_cod,drugname,prodai,route,caseid....
drop all not-Oral component.....
drop dosage imformation....
drop all null data....
drop duplicates in caseid and prodai....
         primaryid    caseid role_cod  drugname                prod_ai route
0        100046942  10004694       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
5        100048206  10004820       PS  DILANTIN              PHENYTOIN  Oral
7        100048622  10004862       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
12       100051352  10005135       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
15       100051382  10005138       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
...            ...       ...      ...       ...                    ...   ...
1943510   99974543   9997454       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
1943518   99975132   9997513       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
1943520   99977523   9997752       PS   LIPITOR   ATORVASTATIN CALCIUM  Oral
1943527   99978615   9997861       PS    TOV

In [17]:
df1 = All_of_pre(df)


drop duplicates in role_cod,drugname,prodai,route,caseid....
drop all not-Oral component.....
drop dosage imformation....
drop all null data....
drop duplicates in caseid and prodai....
drop duplicates in caseid, role_cod, prod_ai....
drop all caseid which don't have PS
find more than one row....
prod_ai to pid and smiles....
         primaryid  ...  route
1934133   39703652  ...   Oral
1934127   39703652  ...   Oral
1934155   40887032  ...   Oral
1934156   40887032  ...   Oral
1934166   40911152  ...   Oral
...            ...  ...    ...
1933589  176107611  ...   Oral
1933598  176107951  ...   Oral
1933599  176107951  ...   Oral
1933612  176108781  ...   Oral
1933611  176108781  ...   Oral

[140971 rows x 6 columns]
dict 0.0
dict 50.0
100.0


In [18]:
df1.to_csv("wow.csv")

In [ ]:
Oral = pd.read_csv("/content/drive/My Drive/Colab Notebooks/oral_PS만 남긴것.csv",low_memory=False,error_bad_lines=False)

In [ ]:
print(Oral['role_cod'].value_counts())
print(len(Oral['caseid'].unique()))
print(Oral['caseid'].max())

PS    148179
SS     85322
C      59469
I       2423
Name: role_cod, dtype: int64
148179
17629161


In [ ]:
Oral_d = Oral.dropna(axis=0)

In [ ]:
print(Oral_d)

        Unnamed: 0    caseid  ...                prod_ai route
0                0  10004694  ...   ATORVASTATIN CALCIUM  Oral
1                2  10004694  ...   ATORVASTATIN CALCIUM  Oral
2                5  10004820  ...              PHENYTOIN  Oral
3                6  10004820  ...              PHENYTOIN  Oral
4                7  10004862  ...   ATORVASTATIN CALCIUM  Oral
...            ...       ...  ...                    ...   ...
295388     1943520   9997752  ...   ATORVASTATIN CALCIUM  Oral
295389     1943521   9997752  ...   ATORVASTATIN CALCIUM  Oral
295390     1943523   9997752  ...   ATORVASTATIN CALCIUM  Oral
295391     1943527   9997861  ...  FESOTERODINE FUMARATE  Oral
295392     1943530   9999811  ...   ATORVASTATIN CALCIUM  Oral

[292581 rows x 6 columns]


In [ ]:
print(len(Oral_d['caseid'].unique()))
grouped = Oral_d.groupby(by =['caseid'])
print(len(grouped))
print(grouped.count())
Only_one= []
not_one=[]
count =0
for caseid,group_data in grouped:
  if (group_data[:].shape[0]) == 1 :
    Only_one.append(group_data[:])
  else :
    not_one.append(group_data[:])
  if group_data[:].shape[0] == 2 :
    count +=1

148179
148179
          Unnamed: 0  role_cod  drugname  prod_ai  route
caseid                                                  
3970365            2         2         2        2      2
3992875            1         1         1        1      1
4015854            1         1         1        1      1
4088703            2         2         2        2      2
4091115            7         7         7        7      7
...              ...       ...       ...      ...    ...
17616774           1         1         1        1      1
17616782           1         1         1        1      1
17616784           1         1         1        1      1
17616875           1         1         1        1      1
17629161           1         1         1        1      1

[148179 rows x 5 columns]


In [ ]:
print(count)

35600


In [ ]:
print(Only_one)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(pd.concat(Only_one))

        Unnamed: 0  ...  route
293948     1934137  ...   Oral
293949     1934143  ...   Oral
293960     1934172  ...   Oral
293961     1934213  ...   Oral
293962     1934215  ...   Oral
...            ...  ...    ...
293941     1934114  ...   Oral
293942     1934116  ...   Oral
293943     1934117  ...   Oral
293944     1934120  ...   Oral
293945     1934122  ...   Oral

[89072 rows x 6 columns]


In [ ]:
OO = pd.concat(Only_one)
NO = pd.concat(not_one)
print(len(OO),len(NO))

89072 203509


In [ ]:
print(len(NO['caseid'].unique()),NO['role_cod'].value_counts())

59107 SS    85322
PS    59107
C     56657
I      2423
Name: role_cod, dtype: int64


In [ ]:
print(len(OO['caseid'].unique()),OO['role_cod'].value_counts())
OO = OO[['caseid','role_cod','drugname','prod_ai','route']]
NO = NO[['caseid','role_cod','drugname','prod_ai','route']]

89072 PS    89072
Name: role_cod, dtype: int64


In [ ]:
OO.to_csv('only_one_row.csv')
NO.to_csv('not_one_row.csv')

In [ ]:
NO_name = NO['prod_ai'].unique()

In [ ]:
print(NO_name,len(NO_name))

['CYCLOSPORINE' 'ENALAPRIL' 'LAMIVUDINE' ... 'PROPYLENE GLYCOL'
 'PERGOLIDE' 'SOLRIAMFETOL'] 2580


여러개의 약물 이름을 가진 것은 252개 있다

In [ ]:
len(NO_name)

2580

In [ ]:
NO_m = NO[4:9]
No_m_t,D = prodai_to_pupchem(NO_m)

dict 0.0
CISAPRIDE
['2769', '6917698', '5311047', '6604902', '44279790']
dict 20.0
CARISOPRODOL
['2576']
dict 40.0
SENNA LEAF
SENNOSIDES
SENNOSIDES A AND B
['5199', '656822', '5199', '54693543']
dict 60.0
.ALPHA.-TOCOPHEROL ACETATE, DL-
ASCORBIC ACID
CYANOCOBALAMIN
FLUORIDE ION
FOLIC ACID
NIACIN
PYRIDOXINE
RIBOFLAVIN
THIAMINE
VITAMIN A
VITAMIN D
['54670067', '5311498', '4289526', '46853873', '70678590', '11970261', '24892734', '25102581', '54605677', '129627537', '139030973', '5460135', '6474318', '25195380', '44176380', '91898871', '118701720', '129627679', '137332389', '146171101', '152743321', '28179', '135398658', '938', '1054', '493570', '1130', '6042', '6202', '445354', '5280795']
dict 80.0
GLUCOSAMINE
['439213']
dict 100.0
dict 100.0
dict 100.0
dict 100.0
dict 100.0


In [ ]:
print(No_m_t)

    caseid  ...                                             SMILES
0  4091115  ...  COC1CN(CCC1NC(=O)C2=CC(=C(C=C2OC)N)Cl)CCCOC3=C...
1  4091115  ...                     CCCC(C)(COC(=O)N)COC(=O)NC(C)C
2  4091115  ...  C1=CC2=C(C(=C1)OC3C(C(C(C(O3)CO)O)O)O)C(=O)C4=...
3  4091115  ...  C([C@@H]([C@@H]1C(=C(C(=O)O1)O)O)O)O, CC1=CC2=...
4  4091115  ...         C([C@@H]1[C@H]([C@@H]([C@H](C(O1)O)N)O)O)O

[5 rows x 7 columns]


In [ ]:
D_q=D

In [ ]:
NO_p,D = prodai_to_pupchem(NO)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
IVACAFTOR
LUMACAFTOR
['16220172', '16678941']
LABETALOL
LABETALOL HYDROCHLORIDE
['3869', '71412']
MAGNESIUM
['5462224']
ERYTHROMYCIN LACTOBIONATE
['71469', '90472388', '13206344', '146158781']
BIFIDOBACTERIUM SPP.
[]
NIPRADILOL
['72006']
dict 20.0
FURSULTIAMINE HYDROCHLORIDE
['9845819']
CHLORPHENESIN CARBAMATE
['2724']
AMLODIPINE BESYLATE
ATORVASTATIN CALCIUM
['60496', '60822', '15378998', '60823', '656846', '45790066', '101610369', '9833468', '11227182', '17589822']
AZITHROMYCIN DIHYDRATE
['3033819', '447043', '9897015']
BENZONATATE
['7699']
ARMODAFINIL
['9690109']
NADOLOL
['39147']
SUCRALFATE
['121494085']
ABACAVIR SULFATE
DOLUTEGRAVIR SODIUM
LAMIVUDINE
['441384', '9843042', '441300', '46216142', '60825']
ACAI
[]
ETANERCEPT
[]
CORTISONE
['222786']
DESOXIMETASONE
['5311067']
HYDROCORTISONE ACETATE
['5744']
CEPHALEXIN
['27447']
CALCIUM CITRATE
VITAMIN D
['13136', '5280795']
PIRFENIDONE
['40632']
DOXYCYCLINE HYCLATE
['54686183', '54671203', '54705095'

In [ ]:
print(D)

{'CYCLOSPORINE': ['5284373, 6435893, 5280754, 62280, 24883466, 5497195, 25246316, 122172945, 123134263', 'CC[C@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)N[C@H](C(=O)N[C@@H](C(=O)N([C@H](C(=O)N([C@H](C(=O)N([C@H](C(=O)N([C@H](C(=O)N1)[C@@H]([C@H](C)C/C=C/C)O)C)C(C)C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C, CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(C(=O)NC(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N1)[C@@H]([C@H](C)C/C=C/C)O)C)C(C)C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C, CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(C(=O)NC(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N1)C(C(C)C/C=C/C)O)C)C(C)C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C, CCC1C(=O)N(CC(=O)N(C(C(=O)NC(C(=O)N(C(C(=O)NC(C(=O)NC(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N(C(C(=O)N1)[C@@H]([C@H](C)CC=CC)O)C)C(C)C)C)CC(C)C)C)CC(C)C)C)C)C)CC(C)C)C)C(C)C)CC(C)C)C)C, CC[C@@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)N([C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N([C@H](C(=O)N([C@H](C(=O)N([C@@H](C(=O)N([C@H](C(=

In [ ]:
NO_new = NO_p.replace('',np.nan,regex=True)
NO_new = NO_new.dropna(axis=0)
print(NO_new)

          caseid  ...                                             SMILES
0        3970365  ...  CC[C@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)...
1        3970365  ...  CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N2C...
2        4088703  ...               C1[C@H](O[C@H](S1)CO)N2C=CC(=NC2=O)N
3        4088703  ...  CC1=C2COC(=O)C2=C(C(=C1OC)C/C=C(\C)/CCC(=O)OCC...
4        4091115  ...  COC1CN(CCC1NC(=O)C2=CC(=C(C=C2OC)N)Cl)CCCOC3=C...
...          ...  ...                                                ...
203504  17610795  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
203505  17610878  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
203506  17610878  ...  CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...
203507  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...
203508  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...

[201120 rows x 7 columns]


In [ ]:
NO_new.to_csv('pub_revise.csv',index=False)

In [ ]:
NO.to_csv('NO.csv',index=False)

In [ ]:
pub_revise = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/pub_revise.csv",low_memory=False,error_bad_lines=False)

In [ ]:
print(pub_revise)

          caseid  ...                                             SMILES
0        3970365  ...  CC[C@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)...
1        3970365  ...  CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N2C...
2        4088703  ...               C1[C@H](O[C@H](S1)CO)N2C=CC(=NC2=O)N
3        4088703  ...  CC1=C2COC(=O)C2=C(C(=C1OC)C/C=C(\C)/CCC(=O)OCC...
4        4091115  ...  COC1CN(CCC1NC(=O)C2=CC(=C(C=C2OC)N)Cl)CCCOC3=C...
...          ...  ...                                                ...
201115  17610795  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
201116  17610878  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
201117  17610878  ...  CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...
201118  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...
201119  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...

[201120 rows x 7 columns]


In [ ]:
pub_drop = pub_revise.drop_duplicates(["caseid","prod_ai"])
print(pub_drop)

          caseid  ...                                             SMILES
0        3970365  ...  CC[C@H]1C(=O)N(CC(=O)N([C@H](C(=O)N[C@H](C(=O)...
1        3970365  ...  CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N2C...
2        4088703  ...               C1[C@H](O[C@H](S1)CO)N2C=CC(=NC2=O)N
3        4088703  ...  CC1=C2COC(=O)C2=C(C(=C1OC)C/C=C(\C)/CCC(=O)OCC...
4        4091115  ...  COC1CN(CCC1NC(=O)C2=CC(=C(C=C2OC)N)Cl)CCCOC3=C...
...          ...  ...                                                ...
201114  17610795  ...  CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...
201115  17610795  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
201116  17610878  ...  C1C[C@H](N(C1)C(=O)[C@H](CCCCN)N[C@@H](CCC2=CC...
201117  17610878  ...  CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...
201118  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...

[164304 rows x 7 columns]


In [ ]:
pub_drop.to_csv("check.csv",index=False)

In [ ]:
print(len(pub_drop['prod_ai']))

164304


In [ ]:
u_name =[]
for name in pub_drop['prod_ai']:
  n = name.split('\\')
  for ns in n:
    u_name.append(ns)
print(len(u_name))

175984


In [ ]:
print(len(set(u_name)))
print(set(u_name))

2098
{'DOXYCYCLINE HYCLATE', 'MOSAPRIDE CITRATE DIHYDRATE', 'ETORICOXIB', 'DEXTROAMPHETAMINE SACCHARATE', 'ZIPRASIDONE HYDROCHLORIDE', 'AMPICILLIN', 'BARNIDIPINE HYDROCHLORIDE', 'CINNARIZINE', 'FLECAINIDE', 'GLUCOSAMINE SULFATE', 'COUMARIN', 'CORTISONE', 'TRIMETHOPRIM', 'IRON', 'PENICILLIN V BENZATHINE', 'ATAZANAVIR SULFATE', 'AMYLASE', 'BENZTROPINE', 'METOPROLOL', 'LANREOTIDE ACETATE', 'BRIVUDINE', 'ERLOTINIB HYDROCHLORIDE', 'CROMOLYN SODIUM', 'NORGESTREL', 'PROPOXYPHENE HYDROCHLORIDE', 'HYDROCHLORIC ACID', 'TIAGABINE HYDROCHLORIDE', 'AZILSARTAN KAMEDOXOMIL', 'QUAZEPAM', 'FERROUS SULFATE ANHYDROUS', 'RISEDRONATE SODIUM HEMI-PENTAHYDRATE', 'DIGOXIN', 'ESOMEPRAZOLE', 'BICTEGRAVIR', 'SUNITINIB', 'NELFINAVIR', 'FOSINOPRIL SODIUM', 'VANCOMYCIN', 'BIOTIN', 'ZIPRASIDONE', 'BERAPROST SODIUM', 'ABACAVIR SUCCINATE', 'BEDAQUILINE', 'LANTHANUM CARBONATE', 'CLOPIDOGREL HYDROCHLORIDE', 'RIBOFLAVIN', 'ACETAZOLAMIDE', 'ATROPINE SULFATE', 'NITISINONE', 'CIMETIDINE', 'ACLIDINIUM BROMIDE', 'MEDRYSONE', 

In [ ]:
new_pd = prod_ai_to_pub_duplicate(pub_drop)

Streaming output truncated to the last 5000 lines.
[17600142, 'SS', 'NEXIUM', 'ESOMEPRAZOLE MAGNESIUM', '86280304', 'CC1=CC(=NC(=C1OC)C)CS(=O)C2=NC3=C([N-]2)C=CC(=C3)OC.CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C([N-]2)C=CC(=C3)OC.[Mg+2]']
[17600142, 'SS', 'NEXIUM', 'ESOMEPRAZOLE MAGNESIUM', '91886157', 'CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C([N-]2)C=CC(=C3)OC.CC1=CN=C(C(=C1OC)C)CS(=O)C2=NC3=C([N-]2)C=CC(=C3)OC.O.[Mg+2]']
name
['FUROSEMIDE']
[17600145, 'PS', 'FUROSEMIDE.', 'FUROSEMIDE', '3440', 'C1=COC(=C1)CNC2=CC(=C(C=C2C(=O)O)S(=O)(=O)N)Cl']
name
['ASPIRIN DL-LYSINE']
[17600145, 'SS', 'KARDEGIC', 'ASPIRIN DL-LYSINE', '9905405', 'CC(=O)OC1=CC=CC=C1C(=O)O.C(CCN)CC(C(=O)O)N']
name
['CANDESARTAN CILEXETIL', 'HYDROCHLOROTHIAZIDE']
[17600145, 'SS', 'HYTACAND', 'CANDESARTAN CILEXETIL', '2540', 'CCOC1=NC2=CC=CC(=C2N1CC3=CC=C(C=C3)C4=CC=CC=C4C5=NNN=N5)C(=O)OC(C)OC(=O)OC6CCCCC6']
[17600145, 'SS', 'HYTACAND', 'HYDROCHLOROTHIAZIDE', '3639', 'C1NC2=CC(=C(C=C2S(=O)(=O)N1)S(=O)(=O)N)Cl']
name
['ACEBUTOLOL HYDROCH

In [ ]:
new_pd = pd.DataFrame(new_pd)
#print(new_pd[4])
new=pd.DataFrame({'caseid':new_pd[0],'role_cod':new_pd[1],'drugname':new_pd[2],'prod_ai':new_pd[3],'CID':new_pd[4],'SMILES':new_pd[5]})

In [ ]:
print(new)
new.to_csv("new.csv",index=False)

          caseid  ...                                             SMILES
0        3970365  ...  CCOC(=O)[C@H](CCC1=CC=CC=C1)N[C@@H](C)C(=O)N2C...
1        4088703  ...               C1[C@H](O[C@H](S1)CO)N2C=CC(=NC2=O)N
2        4088703  ...  CC1=C2COC(=O)C2=C(C(=C1OC)C/C=C(\C)/CCC(=O)OCC...
3        4088703  ...  CC1=C2COC(=O)C2=C(C(=C1OC)C/C=C(\C)/CCC(=O)OCC...
4        4091115  ...  COC1CN(CCC1NC(=O)C2=CC(=C(C=C2OC)N)Cl)CCCOC3=C...
...          ...  ...                                                ...
270344  17610878  ...  CC1([C@@H](N2[C@H](S1)[C@@H](C2=O)NC(=O)[C@@H]...
270345  17610878  ...  C1[C@@H]2N(C1=O)[C@H](/C(=C/CO)/O2)C(=O)[O-].[K+]
270346  17616523  ...  C[C@H]1CC(N(C1)C2=C(C=CC(=N2)N3C=CC(=N3)OCC(C)...
270347  17616523  ...  CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...
270348  17616523  ...  CC(C)(CO)C1=CC2=CC(=C(C=C2N1C[C@H](CO)O)F)NC(=...

[270349 rows x 6 columns]


In [ ]:
"new_pd =[['caseid','role_cod','drugname','prod_ai','route','CID','SMILES']]
j = 0
for index,com in multi_compound.iterrows():
  if (j==0):
    j=j+1
    continue
  if (j*100%len(multi_compound)==0):
    print(j*100%len(multi_compound))
  c = com['CID'].split(',')
  d = com['SMILES'].split(',')
  if (len(c)!=len(d)):
    print("!")
  i = 0
  for name in range(0,len(c)):
    multi=[]  
    multi.append(com['caseid'])
    multi.append(com['role_cod'])
    multi.append(com['drugname'])
    multi.append(com['prod_ai'])
    multi.append(com['route'])
    multi.append(c[name])
    multi.append(d[name])
    i=i+1;
    new_pd.append(multi)
  j=j+1;

In [ ]:
new_pd = pd.DataFrame(new_pd)

In [ ]:
print(new_pd[5])

0               CID
1           5281078
2           6441022
3              2769
4           6917698
            ...    
131835        33613
131836     23665591
131837    134587348
131838     16220172
131839     46199646
Name: 5, Length: 131840, dtype: object


In [ ]:
new_pd.to_csv("check.csv")